<a href="https://colab.research.google.com/github/botkin92/Recommender_systems/blob/lesson_3/HW_webinar_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задание 1. Попытаться ответить на вопросы/выдвинуть гипотезы

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель) - ***Рост прожад на 3-5% за 4 недели относительно прошлых 4 недель (или этого же месяца прошлого года)***.
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше).
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они? - ***Если пользователь зарегистрирован в программе лояльности, то можно вытащить e-mail из базы. Если пользователь регистрировался более 3-5 лет, возможно, указанный e-mail не актуален. Скорее всего лишь малая часть клиентов активно просматривают и пользуются своей e-mail***.
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе? - ***СМС и push-уведомления повысит охват рекомендаций, но частые уведомления могут быть навящими и оттолкнут клиентов вообще что-либо преобретать. Т.е. лучше предлагать не часто и по отработанной эффективной модели. Рекомендации на чеке также могут дать весьма общирный охват клиентов, здесь на первый план встает себестоимость и окупаемость таких рекомендаций, а также техническая возможность выдачи индивидуальных "горячих" рекомендаций***. 
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?) - ***топ-10 рекомендаций или ранжирование - и то, и то нужно пробовать. 10 можно, но не более. При большом количестве предлагаемых товаров теряется эффект "специального предложения" и может выглядеть навящево. 3 мало, 5 оптимально***.
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)? - ***Например, из соображения: популярные товары и так купят, самые дешёвые тоже. Можно предложить товары среднего сегмента, товары собственного производства, новые скидки***.
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера? - ***Окупить расходы + 3-5%***
---
- А точно нужно сортировать по вероятности? - ***Нет, нужно тестировать и анализировать полученный результат***
- Какую метрику использовать? - ***Precision at K***
- Сколько раз в неделю отпрпавляем рассылку? - ***не более 1-2 раз, либо при обновлении модели рекоментаций*** 
- В какое время отправляем рассылку? - ***Не ночью и не утром***
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались? - ***Взять топ-20 рекомендаций и отправлять случайным образом 5-10***.
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными? - ***Нужно,использовать признак категорий*** 
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) - ***Возможно, стоит использовать другую модель***.
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

### Задание 4. Поэкспериментировать с ALS (grid-search)

### 1. Базовое применение (Из методички)

In [1]:
!pip install implicit==0.4.4

     |████████████████████████████████| 1.1 MB 7.1 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3423218 sha256=dec82e44a80ebccd580dfd5d1699e61c8860103960c7bce518815b0381ddbb0a
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# # Функции из 1-ого вебинара
# import os, sys
# 
# module_path = os.path.abspath(os.path.join(os.pardir))
# if module_path not in sys.path:
#     sys.path.append(module_path)
# 
# from metrics import precision_at_k, recall_at_k


In [3]:
"""
Metrics

"""
import numpy as np

def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)

def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_recommended.sum()

def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_bought.sum()


def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[recommended_list <= k]

    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    amount_relevant = len(relevant_indexes)


    sum_ = sum(
        [precision_at_k(recommended_list, bought_list, k=index_relevant + 1) for index_relevant in relevant_indexes])
    return sum_ / amount_relevant



In [4]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [5]:
# data = pd.read_csv('../data/transaction_data.csv')
data = pd.read_csv('MyDrive/Colab Notebooks/Recommender_systems/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [6]:
# item_features = pd.read_csv('../data/product.csv')
item_features = pd.read_csv('MyDrive/Colab Notebooks/Recommender_systems/product.csv')

item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [7]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [9]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [10]:
data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [11]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

# ALS (из методички)

In [13]:
%%time

model = AlternatingLeastSquares(factors=64, 
                                regularization=0.05,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=1,  # 10
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 11.9 s, sys: 11.7 s, total: 23.7 s
Wall time: 22.9 s


In [14]:
recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

In [15]:
[id_to_itemid[rec[0]] for rec in recs]

[999999, 1133018, 5569230, 1106523, 1082185]

In [16]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [17]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

CPU times: user 26 s, sys: 19.9 s, total: 45.9 s
Wall time: 23.6 s


In [18]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.15570065293821983

In [19]:
result.head(2)

,user_id,actual,als
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[965766, 5569374, 878996, 1100972, 1033142]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1106523, 5569327, 892008, 5569230, 951590]"


Используя циклы, обучим модель с различными значениями параметров: регуляризация и количество итераций

In [20]:
iterations_list = [5, 15, 30, 50]
regularization_list = [0.005, 0.01, 0.05, 0.1, 0.5]
factors = [5, 10, 32, 64, 128]

table_results = {
    'iterations': [],
    'regularization': [],
    'factors': [],
    'precision@5': []}

for iter in iterations_list:
  for reg in regularization_list:
    for factor in factors:
        table_results['iterations'].append(iter)
        table_results['regularization'].append(reg)
        table_results['factors'].append(factor)
        model = AlternatingLeastSquares(factors=factor, 
                                    regularization=reg,
                                    iterations=iter, 
                                    calculate_training_loss=True, 
                                    num_threads=1,  # 10
                                    use_gpu=False)
        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)
        result[f'als_iter:{iter}_reg:{reg}_factors:{factor}'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        table_results['precision@5'].append(result.apply(lambda row: precision_at_k(row[f'als_iter:{iter}_reg:{reg}_factors:{factor}'], row['actual']), axis=1).mean())

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [21]:
df =pd.DataFrame(data=table_results)
df.sort_values('precision@5', ascending=False)

,iterations,regularization,factors,precision@5
45,15,0.50,5,0.174787
15,5,0.10,5,0.173983
65,30,0.10,5,0.173882
60,30,0.05,5,0.173782
70,30,0.50,5,0.173682
...,...,...,...,...
64,30,0.05,128,0.141537
94,50,0.10,128,0.141236
84,50,0.01,128,0.141236
89,50,0.05,128,0.140834


Наблюдения:
1. регуляризация и количество итераций влияют на метрику
2. лучший вариант получился с регуляризацией - 0.5, факторов - 15 и количеством итераций - 5. Несколько странно т.к. данные значения являются крайними в листе параметров, возможно стоит добавить еще значения и найти "точку перегиба" для метрики.

In [22]:
result.head(2)

,user_id,actual,als,als_iter:5_reg:0.005_factors:5,als_iter:5_reg:0.005_factors:10,als_iter:5_reg:0.005_factors:32,als_iter:5_reg:0.005_factors:64,als_iter:5_reg:0.005_factors:128,als_iter:5_reg:0.01_factors:5,als_iter:5_reg:0.01_factors:10,...,als_iter:50_reg:0.1_factors:5,als_iter:50_reg:0.1_factors:10,als_iter:50_reg:0.1_factors:32,als_iter:50_reg:0.1_factors:64,als_iter:50_reg:0.1_factors:128,als_iter:50_reg:0.5_factors:5,als_iter:50_reg:0.5_factors:10,als_iter:50_reg:0.5_factors:32,als_iter:50_reg:0.5_factors:64,als_iter:50_reg:0.5_factors:128
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[965766, 5569374, 878996, 1100972, 1033142]","[999999, 1082185, 6534178, 981760, 995242]","[999999, 1082185, 995242, 6534178, 1029743]","[5569374, 1082212, 1005186, 999999, 1082185]","[1100972, 1033142, 867188, 999999, 832678]","[1033142, 979707, 995242, 999999, 1082212]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 995242, 981760, 6534178]",...,"[999999, 1082185, 6534178, 995242, 981760]","[999999, 1082185, 981760, 995242, 885290]","[885290, 1005186, 832678, 999999, 5569374]","[1024306, 1100972, 1033142, 965766, 878996]","[878996, 1033142, 5569374, 1105488, 962568]","[999999, 1082185, 6534178, 995242, 981760]","[999999, 1082185, 981760, 995242, 1127831]","[885290, 832678, 999999, 995242, 1005186]","[1005186, 901062, 878996, 1105488, 1033142]","[878996, 986912, 1033142, 5569374, 1105488]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1106523, 5569327, 892008, 5569230, 951590]","[999999, 1082185, 1029743, 1106523, 883404]","[999999, 1082185, 951590, 8090521, 1029743]","[962229, 951590, 5569230, 883404, 908531]","[5569230, 826249, 910032, 951590, 908531]","[1133018, 1022003, 938700, 1106523, 925514]","[999999, 1082185, 1106523, 1029743, 951590]","[999999, 883404, 1082185, 1029743, 951590]",...,"[999999, 1082185, 1029743, 1106523, 883404]","[999999, 1029743, 1082185, 883404, 8090521]","[951590, 5569230, 883404, 1106523, 8090521]","[1106523, 951590, 5569327, 1133018, 5568378]","[914190, 1106523, 1133018, 908531, 1022003]","[999999, 1082185, 1029743, 1106523, 883404]","[999999, 1029743, 883404, 1082185, 951590]","[951590, 5569230, 1106523, 883404, 962229]","[951590, 5569327, 1106523, 1092026, 1133018]","[914190, 1022003, 908531, 1106523, 1133018]"


In [25]:
df.loc[(df.iterations == 15) & (df.factors == 64)].sort_values('precision@5', ascending=False)
# df.loc[df.iterations == 'Green']

,iterations,regularization,factors,precision@5
43,15,0.100,64,0.158815
38,15,0.050,64,0.157710
33,15,0.010,64,0.156504
48,15,0.500,64,0.153491
28,15,0.005,64,0.153089


In [27]:
df.loc[(df.regularization == 0.05) & (df.factors == 64)].sort_values('precision@5', ascending=False)

,iterations,regularization,factors,precision@5
13,5,0.05,64,0.165645
38,15,0.05,64,0.157710
63,30,0.05,64,0.154596
88,50,0.05,64,0.154395


In [26]:
df.loc[(df.iterations == 15) & (df.regularization == 0.05)].sort_values('precision@5', ascending=False)

,iterations,regularization,factors,precision@5
35,15,0.05,5,0.171974
36,15,0.05,10,0.167052
37,15,0.05,32,0.165645
38,15,0.05,64,0.157710
39,15,0.05,128,0.148468


Рассматривая метрики с различной регуляризацией при фиксированном количестве итераций и факторов, видно что precision@5 линейно не зависит от регуляризации. По крайней мере для данного примера.  
А вот с уменьшением количества итераций precision@5 увеличивается при фиксированном значении регуляризации и факторов.  
При фиксированном значении регуляризации и итераций, видно что количество факторов влияют на метрику precision@5. Чем меньше factors, тем выше метрика.